In [2]:
# install msodbcsql17
%%sh
apt-get install -y unixodbc-dev
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

Reading package lists...
Building dependency tree...
Reading state information...
unixodbc-dev is already the newest version (2.3.9-5ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 51 not upgraded.
OK
Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://packages.microsoft.com/ubuntu/22.04/prod jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launc

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key is deprecated. Manage keyring files in trusted.gpg.d instead (see apt-key(8)).
100   983  100   983    0     0   6780      0 --:--:-- --:--:-- --:--:--  6826
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    88  100    88    0     0   1065      0 --:--:-- --:--:-- --:--:--  1073
W: https://packages.microsoft.com/ubuntu/22.04/prod/dists/jammy/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide

In [3]:
!pip install pyodbc

In [4]:
import pyodbc
import pandas as pd
import os

server=''
user=''
password=''

# Create a database connection
conn=pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER='+server+';' # Specify the server name and port
    'UID='+user+';'
    'PWD='+password+';'
)
print(conn)

In [5]:
# for each fac_type, the node feature is given by 168*5 0 matrix for [[From,To,Keep,許可max,許可avg], [From,To,Keep,許可max,許可avg]....]]
node_feature_A1=[[0 for i in range(5)] for i in range(168)]
node_feature_A2=[[0 for i in range(5)] for i in range(168)]
node_feature_C=[[0 for i in range(5)] for i in range(168)]
node_feature_RM=[[0 for i in range(5)] for i in range(168)]
node_feature_RP=[[0 for i in range(5)] for i in range(168)]
node_feature_S=[[0 for i in range(5)] for i in range(168)]

In [31]:
filename='file_1_mapping_new.csv' # node set file name
inputname='input.csv' # input file name, i.e. nodes that to be queried for node features

## A1

In [6]:
def A1(fac_no, code_no):
    # 產出, 聯單, 貯存
    qstr="""SELECT *
            FROM IMP_3.dbo.Waste_tmp_month_statistics wtms
            WHERE fac_no = '{}'
                AND CodeNo = '{}'
                AND DType = 3 --廢棄物
            ORDER BY AppTime"""
    df=pd.read_sql(qstr.format(fac_no, code_no), conn)

    for index, row in df.iterrows():
        year=int(row['AppTime'][:4])
        month=int(row['AppTime'][-2:])

        # time range: 2010/01 ~ 2023/12
        if year<2010 or year>2023:
            continue

        if row['AppType']=='manu': # 產出
            node_feature_A1[(year-2010)*12+(month-1)][0]=row['app_qty']
        elif row['AppType']=='Dlist': # 聯單
            node_feature_A1[(year-2010)*12+(month-1)][1]=row['app_qty']
        elif row['AppType']=='Store': # 貯存
            node_feature_A1[(year-2010)*12+(month-1)][2]=row['app_qty']


    # 許可max產生, 許可avg產生
    qstr="""SELECT *
            FROM IMP_3.dbo.Waste_Tmp_ClePlanPro wtcpp
            WHERE fac_no = '{}'
	            AND CodeNo = '{}'
	            AND DType = 3 --廢棄物"""
    df=pd.read_sql(qstr.format(fac_no, code_no), conn)

    for i in range(len(node_feature_A1)):
        node_feature_A1[i][3]=df['Pd_qty_mMax'][0] # 許可max產生
        node_feature_A1[i][4]=df['Pd_qty_mAvg'][0] # 許可avg產生

## A2

In [9]:
def A2(FacID, Waste_No):#非列管對象(A2) Input:FacID,Waste_No 將From(產出)、to(營運紀錄)按時序加入node_feature_A2
                       #From與to同為ReuQty(verify on 9/6)
    qstr="""SELECT Reu_No, Fac_Name, FacID, Cle_No, Waste_No, ProductNo, ProductName, ProductQty, ReuQty, Rec_Date
    FROM IMP_1.dbo.Waste_ReuMonReportN_Commit wrmrnc --再利用機構申報月營運紀錄-已正式申報
    WHERE FacID = '{}'
        AND Waste_No = '{}'
    ORDER BY Rec_Date
    """
    df=pd.read_sql(qstr.format(FacID,Waste_No), conn)
    #df_pdc=df["ProductQty"](verify at 9/6)
    df_reuse=df["ReuQty"]
    df_date=df["Rec_Date"]#2022-06-23 datatype:timestamp
    for i in range(len(df_date)):
        year=df_date[i].year-2010
        month=df_date[i].month-1
        #node_feature_A2[year*12+month][0]+= df_pdc[i](verify on 9/6)
        node_feature_A2[year*12+month][0]+= df_reuse[i]#From(產出)
        node_feature_A2[year*12+month][1]+= df_reuse[i]#to(營運紀錄)

## C

In [44]:
def C(fac_no, code_no):
    if C_clean(fac_no): # if fac_no not in (A1 set, A2 set, RM set), 一般清除
        qstr="""SELECT SUM(App_Qty), MONTH(Wcle_date), YEAR(Wcle_date)
                FROM (
                    SELECT wdw.Dlist_no1, Cle_no, Reu_no, Wcle_date, Waste_no, App_Qty
                    FROM IMP_3.dbo.Waste_Dlist_wre1 wdw --事業機構廢棄物回收利用紀錄檔1
                    LEFT JOIN IMP_3.dbo.Waste_Dlist_wre2 wdw2 --事業機構廢棄物回收利用紀錄檔2
                        ON wdw.Dlist_no1 = wdw2.Dlist_no1
                    WHERE wdw.Dlist_no2 = wdw2.Dlist_no2
                        AND wdw.Dlist_no3 = wdw2.Dlist_no3
                        AND Cle_no = '{}'
                        AND Waste_no = '{}'
                    UNION
                    SELECT wdw3.Dlist_no1, Cle_no, Reu_no, Wcle_date, Waste_no, App_Qty
                    FROM IMP_3.dbo.Waste_Dlist_wt1 wdw3 --事業廢棄物處理後三聯單紀錄檔1
                    LEFT JOIN IMP_3.dbo.Waste_Dlist_wt2 wdw4 --事業廢棄物處理後三聯單紀錄檔2
                        ON wdw3.Dlist_no1 = wdw4.Dlist_no1
                    WHERE wdw3.Dlist_no2 = wdw4.Dlist_no2
                        AND wdw3.Dlist_no3 = wdw4.Dlist_no3
                        AND Cle_no = '{}'
                        AND Waste_no = '{}'
                    UNION
                    SELECT FacID AS Dlist_no1, Cle_No AS Cle_no, Reu_No AS Reu_no, Rec_Date AS Wcle_date, Waste_No AS Waste_no, ReuQty AS App_Qty
                    FROM IMP_1.dbo.Waste_ReuMonReportN_Commit wrmrnc --再利用機構申報月營運紀錄-已正式申報
                    WHERE Cle_No = '{}'
                        AND Waste_No = '{}') AS SubQuery
                GROUP BY MONTH(Wcle_date), YEAR(Wcle_date)
                ORDER BY YEAR(Wcle_date), MONTH(Wcle_date)"""
        df=pd.read_sql(qstr.format(fac_no, code_no, fac_no, code_no, fac_no, code_no), conn)

        for index, row in df.iterrows():
            app_qty=row.iloc[0]
            year=int(row.iloc[2])
            month=int(row.iloc[1])

            # time range: 2010/01 ~ 2023/12
            if year<2010 or year>2023:
                continue

            node_feature_C[(year-2010)*12+(month-1)][0]=app_qty
            node_feature_C[(year-2010)*12+(month-1)][1]=app_qty
            node_feature_C[(year-2010)*12+(month-1)][3]=app_qty
            node_feature_C[(year-2010)*12+(month-1)][4]=app_qty
    elif A1_clean(fac_no): # if fac_no is in A1 set, 列管自行清除
        qstr="""SELECT SUM(App_Qty), MONTH(Wcle_date), YEAR(Wcle_date)
                FROM (
                    SELECT wdw.Dlist_no1, Cle_no, Reu_no, Wcle_date, Waste_no, App_Qty
                    FROM IMP_3.dbo.Waste_Dlist_wre1 wdw --事業機構廢棄物回收利用紀錄檔1
                    LEFT JOIN IMP_3.dbo.Waste_Dlist_wre2 wdw2 --事業機構廢棄物回收利用紀錄檔2
                        ON wdw.Dlist_no1 = wdw2.Dlist_no1
                    WHERE wdw.Dlist_no2 = wdw2.Dlist_no2
                        AND wdw.Dlist_no3 = wdw2.Dlist_no3
                        AND wdw.Dlist_no1 = '{}'
                        AND Cle_no = 'C0000000'
                        AND Waste_no = '{}'
                    UNION
                    SELECT wdw3.Dlist_no1, Cle_no, Reu_no, Wcle_date, Waste_no, App_Qty
                    FROM IMP_3.dbo.Waste_Dlist_wt1 wdw3 --事業廢棄物處理後三聯單紀錄檔1
                    LEFT JOIN IMP_3.dbo.Waste_Dlist_wt2 wdw4 --事業廢棄物處理後三聯單紀錄檔2
                        ON wdw3.Dlist_no1 = wdw4.Dlist_no1
                    WHERE wdw3.Dlist_no2 = wdw4.Dlist_no2
                        AND wdw3.Dlist_no3 = wdw4.Dlist_no3
                        AND wdw3.Dlist_no1 = '{}'
                        AND Cle_no = 'C0000000'
                        AND Waste_no = '{}') AS SubQuery
                GROUP BY MONTH(Wcle_date), YEAR(Wcle_date)"""
        df=pd.read_sql(qstr.format(fac_no, code_no, fac_no, code_no), conn)

        for index, row in df.iterrows():
            app_qty=row.iloc[0]
            year=int(row.iloc[2])
            month=int(row.iloc[1])

            # time range: 2010/01 ~ 2023/12
            if year<2010 or year>2023:
                continue

            node_feature_C[(year-2010)*12+(month-1)][0]=app_qty
            node_feature_C[(year-2010)*12+(month-1)][1]=app_qty
            node_feature_C[(year-2010)*12+(month-1)][3]=app_qty
            node_feature_C[(year-2010)*12+(month-1)][4]=app_qty
    elif A2_clean(fac_no): # if fac_no is in A2 set, 非列管自行清除
        qstr="""SELECT SUM(ReuQty), MONTH(Rec_Date), YEAR(Rec_Date)
                FROM (
                    SELECT Reu_No, Fac_Name, FacID, Cle_No, Waste_No, ProductNo, ProductName, ProductQty, ReuQty, Rec_Date
                    FROM IMP_1.dbo.Waste_ReuMonReportN_Commit wrmrnc --再利用機構申報月營運紀錄-已正式申報
                    WHERE FacID = '{}'
                        AND Cle_No = 'C0000000'
                        AND Waste_No = '{}') AS SubQuery
                GROUP BY MONTH(Rec_Date), YEAR(Rec_Date)"""
        df=pd.read_sql(qstr.format(fac_no, code_no), conn)

        for index, row in df.iterrows():
            app_qty=row.iloc[0]
            year=int(row.iloc[2])
            month=int(row.iloc[1])

            # time range: 2010/01 ~ 2023/12
            if year<2010 or year>2023:
                continue

            node_feature_C[(year-2010)*12+(month-1)][0]=app_qty
            node_feature_C[(year-2010)*12+(month-1)][1]=app_qty
            node_feature_C[(year-2010)*12+(month-1)][3]=app_qty
            node_feature_C[(year-2010)*12+(month-1)][4]=app_qty
    elif RM_clean(fac_no): # if fac_no is in RM set, 列管/非列管再利用者清除
        # 注意, 有例外: Cle_no = 'H47A6728', Reu_no = 'H47A6728'
        qstr="""SELECT SUM(App_Qty), MONTH(Wcle_date), YEAR(Wcle_date)
                FROM (
                    SELECT wdw.Dlist_no1, Cle_no, Reu_no, Wcle_date, Waste_no, App_Qty
                    FROM IMP_3.dbo.Waste_Dlist_wre1 wdw --事業機構廢棄物回收利用紀錄檔1
                    LEFT JOIN IMP_3.dbo.Waste_Dlist_wre2 wdw2 --事業機構廢棄物回收利用紀錄檔2
                        ON wdw.Dlist_no1 = wdw2.Dlist_no1
                    WHERE wdw.Dlist_no2 = wdw2.Dlist_no2
                        AND wdw.Dlist_no3 = wdw2.Dlist_no3
                        AND Cle_no = 'C1111111'
                        AND Reu_no = '{}'
                        AND Waste_no = '{}'
                    UNION
                    SELECT wdw3.Dlist_no1, Cle_no, Reu_no, Wcle_date, Waste_no, App_Qty
                    FROM IMP_3.dbo.Waste_Dlist_wt1 wdw3 --事業廢棄物處理後三聯單紀錄檔1
                    LEFT JOIN IMP_3.dbo.Waste_Dlist_wt2 wdw4 --事業廢棄物處理後三聯單紀錄檔2
                        ON wdw3.Dlist_no1 = wdw4.Dlist_no1
                    WHERE wdw3.Dlist_no2 = wdw4.Dlist_no2
                        AND wdw3.Dlist_no3 = wdw4.Dlist_no3
                        AND Cle_no = 'C1111111'
                        AND Reu_no = '{}'
                        AND Waste_no = '{}'
                    UNION
                    SELECT FacID AS Dlist_no1, Cle_No AS Cle_no, Reu_No AS Reu_no, Rec_Date AS Wcle_date, Waste_No AS Waste_no, ReuQty AS App_Qty
                    FROM IMP_1.dbo.Waste_ReuMonReportN_Commit wrmrnc --再利用機構申報月營運紀錄-已正式申報
                    WHERE Cle_No = 'C1111111'
                        AND Reu_No = '{}'
                        AND Waste_No = '{}') AS SubQuery
                GROUP BY MONTH(Wcle_date), YEAR(Wcle_date)"""
        df=pd.read_sql(qstr.format(fac_no, code_no, fac_no, code_no, fac_no, code_no), conn)

        for index, row in df.iterrows():
            app_qty=row.iloc[0]
            year=int(row.iloc[2])
            month=int(row.iloc[1])

            # time range: 2010/01 ~ 2023/12
            if year<2010 or year>2023:
                continue

            node_feature_C[(year-2010)*12+(month-1)][0]=app_qty
            node_feature_C[(year-2010)*12+(month-1)][1]=app_qty
            node_feature_C[(year-2010)*12+(month-1)][3]=app_qty
            node_feature_C[(year-2010)*12+(month-1)][4]=app_qty
    else:
        print('Bug appears!')

## RM

In [12]:
def RM(fac_no, code_no):
    # From
    # 營運紀錄+聯單
    qstr="""SELECT SUM(App_Qty), MONTH(Wcle_date), YEAR(Wcle_date)
            FROM (
                SELECT FacID AS Dlist_no1, Cle_No AS Cle_no, Reu_No AS Reu_no, Rec_Date AS Wcle_date, Waste_No AS Waste_no, ReuQty AS App_Qty
                FROM IMP_1.dbo.Waste_ReuMonReportN_Commit wrmrnc --再利用機構申報月營運紀錄-已正式申報
                WHERE Reu_No = '{}'
                    AND Waste_No = '{}'
                UNION
                SELECT wdw.Dlist_no1, Cle_no, Reu_no, Wcle_date, Waste_no, App_Qty
                FROM IMP_3.dbo.Waste_Dlist_wre1 wdw --事業機構廢棄物回收利用紀錄檔1
                LEFT JOIN IMP_3.dbo.Waste_Dlist_wre2 wdw2 --事業機構廢棄物回收利用紀錄檔2
                    ON wdw.Dlist_no1 = wdw2.Dlist_no1
                WHERE wdw.Dlist_no2 = wdw2.Dlist_no2
                    AND wdw.Dlist_no3 = wdw2.Dlist_no3
                    AND Reu_no = '{}'
                    AND Waste_no = '{}'
                UNION
                SELECT wdw3.Dlist_no1, Cle_no, Reu_no, Wcle_date, Waste_no, App_Qty
                FROM IMP_3.dbo.Waste_Dlist_wt1 wdw3 --事業廢棄物處理後三聯單紀錄檔1
                LEFT JOIN IMP_3.dbo.Waste_Dlist_wt2 wdw4 --事業廢棄物處理後三聯單紀錄檔2
                    ON wdw3.Dlist_no1 = wdw4.Dlist_no1
                WHERE wdw3.Dlist_no2 = wdw4.Dlist_no2
                    AND wdw3.Dlist_no3 = wdw4.Dlist_no3
                    AND Reu_no = '{}'
                    AND Waste_no = '{}') AS SubQuery
            GROUP BY MONTH(Wcle_date), YEAR(Wcle_date)"""
    df=pd.read_sql(qstr.format(fac_no, code_no, fac_no, code_no, fac_no, code_no), conn)

    for index, row in df.iterrows():
        app_qty=row.iloc[0]
        year=int(row.iloc[2])
        month=int(row.iloc[1])

        # time range: 2010/01 ~ 2023/12
        if year<2010 or year>2023:
            continue

        node_feature_RM[(year-2010)*12+(month-1)][0]=app_qty

    # 收受貯存(上個月)
    qstr="""SELECT reu_no, App_datetime, Waste_no, App_qty
            FROM IMP_3.dbo.Waste_ROnsiteStore wrs --再利用者廠內暫存紀錄檔
            WHERE reu_no = '{}'
                AND Waste_no = '{}'
            ORDER BY App_datetime"""
    df=pd.read_sql(qstr.format(fac_no, code_no), conn)

    df['App_datetime']=df['App_datetime'].astype(str)

    for index, row in df.iterrows():
        year=int(row['App_datetime'][:4])
        month=int(row['App_datetime'][5:7])

        # move backward a month
        if month==1:
            year-=1
            month=12
        else:
            month-=1

        # time range: 2010/01 ~ 2023/1
        if year<2010 or year>2023:
            continue

        node_feature_RM[(year-2010)*12+(month-1)][0]+=row['App_qty']


    # To
    # 原料使用
    qstr="""SELECT *
            FROM IMP_3.dbo.Waste_tmp_month_statistics wtms
            WHERE fac_no = '{}'
                AND CodeNo = '{}'
                AND AppType = 'manu' --使用
                AND DType = 1 --原料
            ORDER BY AppTime"""
    df=pd.read_sql(qstr.format(fac_no, code_no), conn)

    for index, row in df.iterrows():
        year=int(row['AppTime'][:4])
        month=int(row['AppTime'][-2:])

        # time range: 2010/01 ~ 2023/12
        if year<2010 or year>2023:
            continue

        node_feature_RM[(year-2010)*12+(month-1)][1]=row['app_qty']


    # Keep
    # 收受貯存
    qstr="""SELECT *
            FROM IMP_3.dbo.Waste_ROnsiteStore wrs --再利用者廠內暫存紀錄檔
            WHERE reu_no = '{}'
                AND Waste_no = '{}'
            ORDER BY App_datetime"""
    df=pd.read_sql(qstr.format(fac_no, code_no), conn)

    df['App_datetime']=df['App_datetime'].astype(str)

    for index, row in df.iterrows():
        year=int(row['App_datetime'][:4])
        month=int(row['App_datetime'][5:7])

        # time range: 2010/01 ~ 2023/1
        if year<2010 or year>2023:
            continue

        node_feature_RM[(year-2010)*12+(month-1)][2]=row['App_qty']


    # 許可max, 許可avg
    # 許可max使用, 許可avg使用
    qstr="""SELECT *
            FROM IMP_3.dbo.Waste_Tmp_ClePlanPro wtcpp
            WHERE fac_no = '{}'
                AND CodeNo = '{}'
                AND DType = 1 --原料"""
    df=pd.read_sql(qstr.format(fac_no, code_no), conn)

    for i in range(len(node_feature_RM)):
        node_feature_RM[i][3]=df['Pd_qty_mMax'][0] # 許可max使用
        node_feature_RM[i][4]=df['Pd_qty_mAvg'][0] # 許可avg使用

## RP

In [15]:
def RP(OrganID, WasteName):#再利用原料(RP) Input:OrganID,WasteName 將From(生產)、To(銷售)、keep(庫存)
    #按時序加入node_feature_RP，並個別加入許可max、許可avg(verify on 9/6)
    qstr="""SELECT *
    FROM IMP.dbo.ReUse_ReuProductStore rurps --產品生產、銷售及庫存申報檔
    WHERE OrganID = '{}'
        AND WasteName LIKE '%{}%'
    ORDER BY YMDate"""
    df=pd.read_sql(qstr.format(OrganID,WasteName), conn)
    df_pdc=df["ProductionQty"]
    df_SaleQty=df["SaleQty"]
    df_StoreQty=df["StoreQty"]
    df_date=df["YMDate"]#201803 datatype:str
    qstr="""SELECT *
    FROM IMP_3.dbo.Waste_Tmp_ClePlanPro wtcpp
    WHERE fac_no = '{}'
        AND DType = 2 --產品"""
    df=pd.read_sql(qstr.format(OrganID), conn)
    df_Pd_qty_mMax=df["Pd_qty_mMax"]
    df_Pd_qty_mAvg=df["Pd_qty_mAvg"]
    for i in range(len(df_date)):
        year=(int(df_date[i][:4]))-2010
        month=(int(df_date[i][4:]))-1
        node_feature_RP[year*12+month][0]= df_pdc[i]#From(產出)
        node_feature_RP[year*12+month][1]= df_SaleQty[i]#to(銷售)
        node_feature_RP[year*12+month][2]= df_StoreQty[i]#keep(庫存)
        node_feature_RP[year*12+month][3]= df_Pd_qty_mMax[0]#許可max
        node_feature_RP[year*12+month][4]= df_Pd_qty_mAvg[0]#許可avg

## S

In [23]:
def S(TargetID, WasteName, ProductName):#銷售對象(S) Input:TargetID,WasteName 將From(銷售紀錄)按時序加入node_feature_S(verify on 9/6)
    qstr="""SELECT *
    FROM IMP.dbo.ReUse_ReuProductSale rurps
    WHERE TargetID = '{}'
        AND WasteName LIKE '%{}%'
        AND ProductName LIKE '%{}%'
    ORDER BY YMDate"""
    df=pd.read_sql(qstr.format(TargetID, WasteName, ProductName), conn)
    df_SaleQty=df["SaleQty"]
    df_date=df["YMDate"]#201803 datatype:str
    for i in range(len(df_date)):
        year=(int(df_date[i][:4]))-2010
        month=(int(df_date[i][4:]))-1
        node_feature_S[year*12+month][0]= df_SaleQty[i]

## For C, to identify '一般清除'

In [37]:
# if fac_no not in (A1 set, A2 set, RM set), 一般清除
def C_clean(fac_no):
    df_node=pd.read_csv(filename)

    options=['A1', 'A2', 'RM']
    df_node_A1A2RM=df_node[df_node['fac_type'].isin(options)]

    for index, row in df_node_A1A2RM.iterrows():
        if row['fac_no']==fac_no:
            return False # not 一般清除
    return True # 一般清除

## For C, to identify '列管自行清除'

In [38]:
# if fac_no is in A1 set, 列管自行清除
def A1_clean(fac_no):
    df_node=pd.read_csv(filename)

    options=['A1']
    df_node_A1=df_node[df_node['fac_type'].isin(options)]

    for index, row in df_node_A1.iterrows():
        if row['fac_no']==fac_no:
            return True # 列管自行清除
    return False # not 列管自行清除

## For C, to identify '非列管自行清除'

In [39]:
# if fac_no is in A2 set, 非列管自行清除
def A2_clean(fac_no):
    df_node=pd.read_csv(filename)

    options=['A2']
    df_node_A2=df_node[df_node['fac_type'].isin(options)]

    for index, row in df_node_A2.iterrows():
        if row['fac_no']==fac_no:
            return True # 非列管自行清除
    return False # not 非列管自行清除

## For C, to identify '列管/非列管再利用者清除'

In [40]:
# if fac_no is in RM set, 列管再利用者清除 or 非列管再利用者清除
def RM_clean(fac_no):
    df_node=pd.read_csv(filename)

    options=['RM']
    df_node_RM=df_node[df_node['fac_type'].isin(options)]

    for index, row in df_node_RM.iterrows():
        if row['fac_no']==fac_no:
            return True # 列管/非列管再利用者清除
    return False # not 列管/非列管再利用者清除

## Main Function

In [45]:
df_node=pd.read_csv(inputname)

node_feature=[]

for index, row in df_node.iterrows():
    if row['fac_type']=='A1':
        A1(row['fac_no'], row['code_no'])
        node_feature.append(node_feature_A1)
        node_feature_A1=[[0 for i in range(5)] for i in range(168)] # re-initialize
    elif row['fac_type']=='A2':
        A2(row['fac_no'], row['code_no'])
        node_feature.append(node_feature_A2)
        node_feature_A2=[[0 for i in range(5)] for i in range(168)] # re-initialize
    elif row['fac_type']=='C':
        C(row['fac_no'], row['code_no'])
        node_feature.append(node_feature_C)
        node_feature_C=[[0 for i in range(5)] for i in range(168)] # re-initialize
    elif row['fac_type']=='RM':
        RM(row['fac_no'], row['code_no'])
        node_feature.append(node_feature_RM)
        node_feature_RM=[[0 for i in range(5)] for i in range(168)] # re-initialize
    elif row['fac_type']=='RP':
        RP(row['fac_no'], row['code_no'])
        node_feature.append(node_feature_RP)
        node_feature_RP=[[0 for i in range(5)] for i in range(168)] # re-initialize
    elif row['fac_type']=='S':
        S(row['fac_no'], 'R-0201', row['code_no'][:6])
        node_feature.append(node_feature_S)
        node_feature_S=[[0 for i in range(5)] for i in range(168)] # re-initialize
    else:
        print('Invalid fac_type!')

df=pd.DataFrame(node_feature)

df.to_csv('node_feature_format.csv', index=False)

<ipython-input-6-e8657359b818>:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(qstr.format(fac_no, code_no), conn)
<ipython-input-6-e8657359b818>:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(qstr.format(fac_no, code_no), conn)
<ipython-input-44-767d880b4257>:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(qstr.format(fac_no, code_no, fac_no, code_no, fac_no, code_no), conn)
<ipython-input-44-767d880b4257>:91: UserWarning: pandas only supports SQLAlchemy connectable (engin

In [47]:
df

,0,1,2,3,4,5,6,7,8,9,...,158,159,160,161,162,163,164,165,166,167
0,"[0, 0, 0.0, 40.0, 20.0]","[0, 0, 0.0, 40.0, 20.0]","[0, 0, 0.0, 40.0, 20.0]","[2.74, 0, 0.0, 40.0, 20.0]","[2.94, 0, 0.0, 40.0, 20.0]","[3.47, 0, 0.0, 40.0, 20.0]","[2.76, 0, 0.0, 40.0, 20.0]","[2.6, 0, 0.0, 40.0, 20.0]","[3.41, 0, 0.0, 40.0, 20.0]","[3.0, 0, 0.0, 40.0, 20.0]",...,"[0, 0, 0, 40.0, 20.0]","[0, 0, 0, 40.0, 20.0]","[0, 0, 0, 40.0, 20.0]","[0, 0, 0, 40.0, 20.0]","[0, 0, 0, 40.0, 20.0]","[0, 0, 0, 40.0, 20.0]","[0, 0, 0, 40.0, 20.0]","[0, 0, 0, 40.0, 20.0]","[0, 0, 0, 40.0, 20.0]","[0, 0, 0, 40.0, 20.0]"
1,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]",...,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
2,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]",...,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
3,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]",...,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
4,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]",...,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
5,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]",...,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
6,"[0, 0, 0, 250.0, 125.0]","[0, 0, 0, 250.0, 125.0]","[0, 0, 0, 250.0, 125.0]","[0, 0, 0, 250.0, 125.0]","[0, 0, 0, 250.0, 125.0]","[0, 0, 0, 250.0, 125.0]","[0, 0, 0, 250.0, 125.0]","[0, 0, 0, 250.0, 125.0]","[0, 0, 0, 250.0, 125.0]","[0, 0, 0, 250.0, 125.0]",...,"[0, 0, 0, 250.0, 125.0]","[0, 0, 0, 250.0, 125.0]","[0, 0, 0, 250.0, 125.0]","[0, 0, 0, 250.0, 125.0]","[0, 0, 0, 250.0, 125.0]","[0, 0, 0, 250.0, 125.0]","[0, 0, 0, 250.0, 125.0]","[0, 0, 0, 250.0, 125.0]","[0, 0, 0, 250.0, 125.0]","[0, 0, 0, 250.0, 125.0]"
7,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]",...,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
8,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]",...,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"


In [ ]:
A1('H4604228', 'R-0201') # 華膳空廚, 廢塑膠
A2('61604805', 'R-0201') # 南投縣政府, 廢塑膠
RM('H47A6728', 'R-0201') # 柯富伊, 廢塑膠
RP('H47A6728', 'R-0201') # 柯富伊, 廢塑膠
S('27260485', 'R-0201', '220099') # 柯富伊二廠, 廢塑膠, 其他未列名塑膠製品

In [26]:
node_feature_A1

[[0, 0, 0.0, 40.0, 20.0],
 [0, 0, 0.0, 40.0, 20.0],
 [0, 0, 0.0, 40.0, 20.0],
 [2.74, 0, 0.0, 40.0, 20.0],
 [2.94, 0, 0.0, 40.0, 20.0],
 [3.47, 0, 0.0, 40.0, 20.0],
 [2.76, 0, 0.0, 40.0, 20.0],
 [2.6, 0, 0.0, 40.0, 20.0],
 [3.41, 0, 0.0, 40.0, 20.0],
 [3.0, 0, 0.0, 40.0, 20.0],
 [3.53, 0, 0.0, 40.0, 20.0],
 [3.37, 0, 0.0, 40.0, 20.0],
 [3.4, 0, 0.0, 40.0, 20.0],
 [2.9, 0, 0.0, 40.0, 20.0],
 [33.53, 0, 0.0, 40.0, 20.0],
 [1.68, 0, 0.0, 40.0, 20.0],
 [2.02, 0, 0.0, 40.0, 20.0],
 [1.11, 0, 0.0, 40.0, 20.0],
 [1.54, 0, 0.0, 40.0, 20.0],
 [1.61, 0, 0.0, 40.0, 20.0],
 [1.71, 0, 0.0, 40.0, 20.0],
 [18.2, 0, 0.0, 40.0, 20.0],
 [1.52, 0, 0.0, 40.0, 20.0],
 [1.82, 0, 0.0, 40.0, 20.0],
 [1.69, 0, 0.0, 40.0, 20.0],
 [1.44, 0, 0.0, 40.0, 20.0],
 [1.51, 0, 0.0, 40.0, 20.0],
 [1.75, 0, 0.0, 40.0, 20.0],
 [1.03, 0, 0.0, 40.0, 20.0],
 [1.14, 0, 0.0, 40.0, 20.0],
 [1.2, 0, 0.0, 40.0, 20.0],
 [1.65, 0, 0.0, 40.0, 20.0],
 [1.18, 0, 0.0, 40.0, 20.0],
 [1.1, 0, 0.0, 40.0, 20.0],
 [1.08, 0, 0.0, 40.0, 20.0],

In [27]:
node_feature_A2

[[0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0,

In [28]:
node_feature_RM

[[0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 250.0, 125.0],
 [0, 0, 0, 2

In [29]:
node_feature_RP

[[0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0,

In [30]:
node_feature_S

[[0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0,

In [32]:
import pandas as pd

In [33]:
my_3d_list = [
    [[1, 2, 3], [4, 5, 6]],
    [[7, 8, 9], [10, 11, 12]]
]

In [35]:
df = pd.DataFrame(my_3d_list)

In [36]:
df

,0,1
0,"[1, 2, 3]","[4, 5, 6]"
1,"[7, 8, 9]","[10, 11, 12]"
